Segmentación del mercado de adolecentes 
===

* *60 min* | Ultima modificación: Junio 22, 2019

En este tutorial se aplica el algoritmo K-means para clasificar un grupo de adolecentes con base en sus intéreses, con el fin de diseñar estrategias publicitarias y servicios encaminados a cada grupo de interés.

## Descripción del problema

Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.   

En este problema se desea determina que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Estas variables corresponden a palabras que pueden asociarse a un interés de la poblaión analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; adicionalmente, dentro de estas variables se incluye  información como el sexo, la edad y la cantidad de contactos de la persona. 

## Preparación

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
%matplotlib inline
%load_ext rpy2.ipython

## Carga de los datos

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv",
    sep = ',',           
    thousands = None,    
    decimal = '.',       
    encoding='latin-1') 

df.head(10)

,gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,NaN,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1
5,2006,F,NaN,142,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
6,2006,F,18.930,72,0,0,0,0,0,0,...,0,2,0,0,2,0,0,0,0,0
7,2006,M,18.322,17,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2006,F,19.055,52,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2006,F,18.708,39,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## Análisis exploratorio

In [3]:
##
## Número de registros
##
len(df)

30000

In [4]:
##
## Valores diferentes de NA
##
df.gender.value_counts()

F    22054
M     5222
Name: gender, dtype: int64

In [5]:
##
## Número de registros con NA
##
df.gender.isna().sum()

2724

In [6]:
##
## La muestra contiene un rango de edades 
## por fuera de la población de interés
##
df.age.describe()

count    24914.000000
mean        17.993950
std          7.858054
min          3.086000
25%         16.312000
50%         17.287000
75%         18.259000
max        106.927000
Name: age, dtype: float64

In [7]:
##
## Se analizan los rangos de las variables 
## para determinar si hay datos por fuera de sus 
## rangos válidos. La variable `edad` contiene
## datos por fuera de la población de interés
##
df.age.describe()

count    24914.000000
mean        17.993950
std          7.858054
min          3.086000
25%         16.312000
50%         17.287000
75%         18.259000
max        106.927000
Name: age, dtype: float64

In [8]:
## cantidad de nulos en la columna age
df.age.isnull().sum()

5086

In [9]:
##
## Se seleccionan las personas entre 13 y 20 años y 
## se descartan las demás observaciones
##
df['age'] = df['age'].map(lambda x: np.nan if x <  13 else x)
df['age'] = df['age'].map(lambda x: np.nan if x >= 20 else x)

##
## Se verifica la variable edad en los registros de las
## personas en la población de interés.
##
df.age.describe()

count    24477.000000
mean        17.252429
std          1.157465
min         13.027000
25%         16.304000
50%         17.265000
75%         18.220000
max         19.995000
Name: age, dtype: float64

In [10]:
##
## Se crean nuevas variables numéricas a partir 
## de información categórica
##
df['female'] = df['gender'].map(lambda x: 1 if x == 'F' else 0)
df['no_gender'] = df['gender'].map(lambda x: 1 if pd.isnull(x) else 0)

## Cantidad de hombres y mujeres en la muestra.
df.gender.value_counts()

F    22054
M     5222
Name: gender, dtype: int64

In [11]:
##
## Número de registros con NA
##
df.gender.isna().sum()

2724

In [12]:
##
## Cantidad de hombres y mujeres en la muestra
## 0=Male, 1=Female
##
df.female.value_counts()

1    22054
0     7946
Name: female, dtype: int64

In [13]:
##
## Cantidad de patrones para los que se 
## conoce y no se conoce el genero
##
df.no_gender.value_counts()

0    27276
1     2724
Name: no_gender, dtype: int64

In [14]:
##
## Edad sin tener en cuenta los datos faltantes
##
df.age.mean()

17.252428933284307

In [15]:
##
## Se calcula una tabla para determinar 
## la edad promedio por año de graduación
##
df.groupby("gradyear")['age'].mean()

gradyear
2006    18.655858
2007    17.706172
2008    16.767701
2009    15.819573
Name: age, dtype: float64

In [16]:
##
## Los valores faltantes se llenan con el promedio
## de acuerdo con el año de graduación
##
ave_age = df.groupby("gradyear")['age'].mean()
df.age = [x if not pd.isnull(x) else ave_age[y] for x, y in zip(df.age, df.gradyear)]
df.age.describe()

count    30000.000000
mean        17.237326
std          1.141821
min         13.027000
25%         16.282000
50%         17.238000
75%         18.212000
max         19.995000
Name: age, dtype: float64

## Entrenamiento del modelo

In [17]:
##
## Se separa la información de las palabras relacionadas
## con los intereses del resto de la información
##
interests = df.iloc[:,4:].copy()
interests.head()

,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,...,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs,female,no_gender
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,1,0,1


In [18]:
##
## Se escala para eliminar problemas asociados
## a la medida de los datos.
##
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
interests_z = scaler.fit_transform(interests)
interests_z

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08333333, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [19]:
##
## Se usa el algoritmo para determinar los centros de 5 grupos
##
from sklearn.cluster import KMeans
df_clusters = KMeans(n_clusters=5, random_state=1).fit(interests_z)

## Evaluación del modelo

In [20]:
##
## Número de patrones asignados a cada cluster
##
(pd.DataFrame(df_clusters.predict(interests_z)))[0].value_counts()

3    16472
0     5222
1     3095
2     2724
4     2487
Name: 0, dtype: int64

In [21]:
##
## Frecuencia de las palabras en cada cluster
##
df_clusters.cluster_centers_

array([[ 1.31415167e-02,  2.98991446e-02,  7.78047293e-03,
         5.40698853e-04,  2.47578924e-03,  2.57595038e-03,
         9.57487553e-04,  1.69714669e-02,  6.88114388e-03,
         1.66443253e-02,  3.95761522e-03,  1.40767468e-03,
         4.57466275e-03,  5.18958254e-03,  2.26850897e-03,
         6.85561088e-03,  5.32130961e-03,  4.45666934e-03,
         1.08196093e-02,  1.20461053e-02,  5.41283470e-03,
         4.44796490e-03,  4.16826248e-03,  2.48946764e-03,
         4.88059871e-03,  3.42567769e-03,  7.43736509e-05,
         1.13302694e-02,  4.43926047e-03,  9.02432018e-03,
         3.63845270e-03,  2.84852547e-03,  8.10382647e-03,
         7.44104612e-03,  9.40731520e-03,  3.44695519e-03,
         8.94839758e-14, -1.47104551e-15],
       [ 1.92484083e-02,  2.93514622e-02,  1.33688237e-02,
         1.61293394e-02,  2.26379318e-02,  8.69892125e-03,
         6.43142333e-02,  8.33603108e-03,  8.61120104e-03,
         1.56738966e-02,  4.43509226e-02,  3.19753739e-03,
         1.14

**Actividad.---** Haga un heatmap para la interpretación de los grupos.

**Actividad.---** Cuál de los clusters anteriores tiene más interés en los deportes?

**Actividad.---** Cuál es el cluster de las princesas?

**Actividad.---** Cuál cluster representa las personas que no hablan o no han posteado sobre sus intereses?

## Análisis del modelo

In [22]:
##
## Se asigna a cada ejemplo de los datos
## el cluster al que pertenece
##
df["cluster"] = df_clusters.predict(interests_z).tolist()

##
## clusters a los que pertenecen los primeros cinco patrones
##
df[["cluster", "gender", "age", "friends"]].head(5)

,cluster,gender,age,friends
0,0,M,18.982,7
1,1,F,18.801,0
2,0,M,18.335,69
3,3,F,18.875,0
4,2,NaN,18.995,10


In [23]:
##
## Características demográficas de los clusters.
## Edad por cluster.
##
df.groupby("cluster")['age'].mean()

cluster
0    17.432631
1    17.052040
2    17.269795
3    17.229694
4    17.072812
Name: age, dtype: float64

In [24]:
## 
## Promedio de mujeres por cluster
##
#
# cluster	female
#       1 0.8381171
#       2 0.7250000
#       3 0.8378198
#       4 0.8027079
#       5 0.6994515
#
df.groupby("cluster")['female'].mean()

cluster
0    0
1    1
2    0
3    1
4    1
Name: female, dtype: int64

In [25]:
for a,b in df.groupby("cluster"):
    print(b['female'].sum())

0
3095
0
16472
2487


In [26]:
##
## Cantidad promedio de amigos por cluster
##
df.groupby("cluster")['friends'].mean()

cluster
0    24.926465
1    39.572536
2    26.139134
3    30.256982
4    33.431846
Name: friends, dtype: float64

**Actividad.---** Al analizar las distintas variables presentadas, qué puede inferir de cada cluster?

**Actividad.---** Repita este ejercicio usando clustering jerárquico.

---

## Apendice --- Solución en R

In [27]:
%%R 

df <- read.csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv", 
    stringsAsFactors = TRUE)

##
## Cantidad de registros por genero.
## La suma no es igual a los 30.000 
## registros de la muestra
##
table(df$gender)


    F     M 
22054  5222 


In [28]:
%%R
##
## La función `table` excluye los NA.
## A continuación se tienen en cuenta en el conteo
##
table(df$gender, useNA = "ifany")


    F     M  <NA> 
22054  5222  2724 


In [29]:
%%R
##
## Se analizan los rangos de las variables 
## para determinar si hay datos por fuera de sus 
## rangos válidos. La variable `edad` contiene
## datos por fuera de la población de interés
##
summary(df$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  3.086  16.312  17.287  17.994  18.259 106.927    5086 


In [30]:
%%R
##
## Se seleccionan las personas entre 13 y 20 años y 
## se descartan las demás observaciones
##
df$age <- ifelse(df$age >= 13 & df$age < 20, df$age, NA)

##
## Se verifica nuevament la variable edad en los registros de las
## personas en la población de interés.
##
summary(df$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  13.03   16.30   17.27   17.25   18.22   20.00    5523 


In [31]:
%%R
##
## Se crean nuevas variables numéricas a partir 
## de información categórica
##
df$female <- ifelse(df$gender == "F" & !is.na(df$gender), 1, 0)

##
## Se crea una nueva columna para indicar que 
## no hay información del sexo de la persona
##
df$no_gender <- ifelse(is.na(df$gender), 1, 0)

##
## Cantidad de hombres y mujeres en la muestra.
##
table(df$gender, useNA = "ifany")


    F     M  <NA> 
22054  5222  2724 


In [32]:
%%R
##
## Cantidad de hombres y mujeres en la muestra
## 1=Female
##
table(df$female, useNA = "ifany")


    0     1 
 7946 22054 


In [33]:
%%R
##
## Cantidad de patrones para los que se 
## conoce y no se conoce el genero
##
table(df$no_gender, useNA = "ifany")


    0     1 
27276  2724 


In [34]:
%%R
##
## La muestra contiene datos faltantes por lo
## que el promdio de la edad es <NA>
##
mean(df$age)

[1] NA


In [35]:
%%R
##
## Se computa la media de la edad sin 
## tener en cuenta los datos faltantes
##
mean(df$age, na.rm = TRUE)

[1] 17.25243


In [36]:
%%R
##
## Se calcula una tabla para determinar 
## la edad promedio por año de graduación
##
aggregate(data = df, age ~ gradyear, mean, na.rm = TRUE)

  gradyear      age
1     2006 18.65586
2     2007 17.70617
3     2008 16.76770
4     2009 15.81957


In [37]:
%%R
##
## 
## Los valores faltantes en la edad se llenan 
## con el promedio de acuerdo con el año de graduación
ave_age <- ave(df$age, 
               df$gradyear, 
               FUN = function(x) mean(x, na.rm = TRUE))
                   
df$age <- ifelse(is.na(df$age), ave_age, df$age)
summary(df$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  13.03   16.28   17.24   17.24   18.21   20.00 


In [38]:
%%R
##
## Se separa la información de las palabras relacionadas
## con los intereses del resto de la información
## y se escala para eliminar problemas asociados
## a la medida de los datos.
##
interests <- df[5:40]
interests_z <- as.data.frame(lapply(interests, scale))

##
## Se usa el algoritmo para determinar los centros de 5 grupos
##
set.seed(2345)
df_clusters <- kmeans(interests_z, 5)

##
## Número de patrones asignados a cada cluster
##
str(df_clusters$size)

 int [1:5] 871 600 5981 1034 21514


In [39]:
%%R
##
## Frecuencia de las palabras en cada cluster
##
df_clusters$centers

   basketball   football      soccer    softball  volleyball    swimming
1  0.16001227  0.2364174  0.10385512  0.07232021  0.18897158  0.23970234
2 -0.09195886  0.0652625 -0.09932124 -0.01739428 -0.06219308  0.03339844
3  0.52755083  0.4873480  0.29778605  0.37178877  0.37986175  0.29628671
4  0.34081039  0.3593965  0.12722250  0.16384661  0.11032200  0.26943332
5 -0.16695523 -0.1641499 -0.09033520 -0.11367669 -0.11682181 -0.10595448
  cheerleading    baseball      tennis      sports        cute          sex
1    0.3931445  0.02993479  0.13532387  0.10257837  0.37884271  0.020042068
2   -0.1101103 -0.11487510  0.04062204 -0.09899231 -0.03265037 -0.042486141
3    0.3303485  0.35231971  0.14057808  0.32967130  0.54442929  0.002913623
4    0.1856664  0.27527088  0.10980958  0.79711920  0.47866008  2.028471066
5   -0.1136077 -0.10918483 -0.05097057 -0.13135334 -0.18878627 -0.097928345
         sexy         hot      kissed       dance        band    marching
1  0.11740551  0.41389104  0.067

### Análisis del modelo

In [40]:
%%R
str(df_clusters)

List of 9
 $ cluster     : int [1:30000] 5 3 5 5 4 5 1 5 5 3 ...
 $ centers     : num [1:5, 1:36] 0.16 -0.092 0.528 0.341 -0.167 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:5] "1" "2" "3" "4" ...
  .. ..$ : chr [1:36] "basketball" "football" "soccer" "softball" ...
 $ totss       : num 1079964
 $ withinss    : num [1:5] 56114 36039 419433 181973 248881
 $ tot.withinss: num 942439
 $ betweenss   : num 137525
 $ size        : int [1:5] 871 600 5981 1034 21514
 $ iter        : int 5
 $ ifault      : int 0
 - attr(*, "class")= chr "kmeans"


In [41]:
%%R
##
## Se asigna a cada ejemplo de los datos
## el cluster al que pertenece
##
df$cluster <- df_clusters$cluster

##
## clusters a los que pertenecen los primeros cinco patrones
##
df[1:5, c("cluster", "gender", "age", "friends")]

  cluster gender    age friends
1       5      M 18.982       7
2       3      F 18.801       0
3       5      M 18.335      69
4       5      F 18.875       0
5       4   <NA> 18.995      10


In [42]:
%%R
##
## Características demográficas de los clusters.
## Edad por cluster.
##
aggregate(data = df, age ~ cluster, mean)

  cluster      age
1       1 16.86497
2       2 17.39037
3       3 17.07656
4       4 17.11957
5       5 17.29849


In [43]:
%%R
## 
## Promedio de mujeres por cluster
##
aggregate(data = df, female ~ cluster, mean)

  cluster    female
1       1 0.8381171
2       2 0.7250000
3       3 0.8378198
4       4 0.8027079
5       5 0.6994515


In [44]:
%%R
##
## Cantidad promedio de amigos por cluster
##
aggregate(data = df, friends ~ cluster, mean)

  cluster  friends
1       1 41.43054
2       2 32.57333
3       3 37.16185
4       4 30.50290
5       5 27.70052
